In [190]:
from utils import *
import mrf
import llm
from importlib import reload
from evaluate import *
import utils
import evaluate
reload(evaluate)

<module 'evaluate' from '/Users/shepardxia/Desktop/directory/commonsense/evaluate.py'>

In [139]:
with open('./example_files/example_question.json', 'r') as file:
    question = json.load(file)

In [209]:
reload(mrf)
reload(llm)
reload(utils)
import os
#from utils import *
import datetime
from evaluate import *

class Commonsense():
    def __init__(self, verbose=False):
        
        self.infer_hist = {}
        self.vebose = verbose

        self.mrf = mrf.Brute(verbose=self.vebose)
        self.agent = llm.LLM(verbose=self.vebose)


    def infer(self, question, run_mrf=False):
        self.mrf = mrf.Brute(verbose=self.vebose)
        self.agent = llm.LLM(verbose=self.vebose)

        self.agent.set_question(question)
        record = self.agent.big_button().copy()

        
        if run_mrf:
            mrf_json = record['mrf']['json']
            mrf_query = record['mrf']['query']
            build_ret = self.mrf.from_json(entire_json=mrf_json)
            self.mrf.update(10000, slack=False)
            res = self.mrf.query(mrf_query)
            record['mrf']['probability'] = res

        #self.infer_hist[question] = record.copy()
        
        return record.copy()

    def infer_from_cache(self, js):
        # build and run mrf from the cached json file
        return 'NotImplemented'


class evaluation():
    def __init__(self, repeat=1, verbose=True, identifier='trial'):
        self.data = None
        # use timestamp as the identifier if None
        self.identifier = 'Experiments/' + identifier
        self.exp_folder = None
        self.records = {}
        self.repeat = repeat

        self.verbose = verbose
        self.infer = Commonsense(verbose=self.verbose)

    

    def load_folder(self, folder_path):

        # replace string "Dataset" with identifier
        self.exp_folder = folder_path.replace('Dataset', self.identifier)
        # create the folder if not exist
        if not os.path.exists(self.exp_folder):
            os.makedirs(self.exp_folder)

        # list the files in the folder
        files = os.listdir(folder_path)
        if folder_path in self.records:
            print('Folder already loaded, duplicate?')
        self.records[folder_path] = {}
        # load the questions
        for filename in files:
            # check if is a json file
            if filename.endswith('.csv'):
                self.data = utils.DatasetQ(path=folder_path + '/' + filename)
            elif filename.endswith('_q.json'):
                with open(folder_path + '/' + filename, 'r') as f:
                    self.records[folder_path][filename] = {}
                    self.records[folder_path][filename]['question'] = json.load(f)


    def run_all(self, folder_path, run_mrf=True):
        self.load_folder(folder_path)
        for filename, question in self.records[folder_path].items():
            ground = self.data.marg(question['question'])
            self.records[folder_path][filename]['ground'] = ground
            for i in range(self.repeat):
                record = self.infer.infer(question['question'], run_mrf=run_mrf)
                self.records[folder_path][filename]['run_' + str(i)] = record
            # save the run results
            with open(self.exp_folder + '/' + filename, 'w') as f:
                json.dump(self.records[folder_path][filename], f)


In [213]:
folds[8:]

['./Dataset/bin1/Salem', './Dataset/bin1/Denver', './Dataset/bin1/FortWorth']

In [216]:
unique_id = str(datetime.datetime.now().strftime('%m%d_%H%M'))
print(unique_id)
for fold in folds[9:]:
    print("Running fold: ", fold)
    test = evaluation(repeat=1, verbose=False, identifier=unique_id)
    test.run_all(fold, run_mrf=True)

0611_1832
Running fold:  ./Dataset/bin1/Denver


KeyboardInterrupt: 

In [204]:
js

{'Variables': [{'Name': 'P',
   'Value': ['$0 - $50',
    '$51 - $100',
    '$101 - $200',
    '$201 - $500',
    '$501 and above']},
  {'Name': 'G', 'Value': ['1', '2', '3-4', '5-6', '7+']},
  {'Name': 'L',
   'Value': ['Beach', 'City Center', 'Suburban', 'Rural', 'Near Attractions']},
  {'Name': 'T',
   'Value': ['Studio', 'Apartment', 'House', 'Condo', 'Unique Stay']},
  {'Name': 'A', 'Value': ['Basic', 'Standard', 'Premium', 'Luxury', 'None']},
  {'Name': 'S',
   'Value': ['Peak', 'Off-Peak', 'Holiday', 'Weekend', 'Weekday']}],
 'Constraints': [{'Target': [{'Name': 'P',
     'Value': ['$0 - $50',
      '$51 - $100',
      '$101 - $200',
      '$201 - $500',
      '$501 and above']}],
   'Condition': [],
   'Probability': [0.1, 0.25, 0.35, 0.2, 0.1]},
  {'Target': [{'Name': 'G', 'Value': ['1', '2', '3-4', '5-6', '7+']}],
   'Condition': [],
   'Probability': [0.15, 0.4, 0.3, 0.1, 0.05]},
  {'Target': [{'Name': 'L',
     'Value': ['Beach',
      'City Center',
      'Suburban',
     

In [208]:
reload(mrf)
mm = mrf.Brute(verbose=True)
js = test.infer.agent.record['mrf']['json'].copy()
mm.from_json(entire_json=js)


Variable:  P
Value:  $0 - $50, $51 - $100, $101 - $200, $201 - $500, $501 and above 


Variable:  G
Value:  1, 2, 3-4, 5-6, 7+ 


Variable:  L
Value:  Beach, City Center, Suburban, Rural, Near Attractions 


Variable:  T
Value:  Studio, Apartment, House, Condo, Unique Stay 


Variable:  A
Value:  Basic, Standard, Premium, Luxury, None 


Variable:  S
Value:  Peak, Off-Peak, Holiday, Weekend, Weekday 

ERROR: Value not found Beach
T Beach
Target Value not found in batch translate


TypeError: cannot unpack non-iterable NoneType object

{'Variables': [{'Name': 'P',
   'Value': ['$0 - $50',
    '$51 - $100',
    '$101 - $200',
    '$201 - $500',
    '$501 and above']},
  {'Name': 'G', 'Value': ['1', '2', '3-4', '5-6', '7+']},
  {'Name': 'L',
   'Value': ['Beach', 'City Center', 'Suburban', 'Rural', 'Near Attractions']},
  {'Name': 'T',
   'Value': ['Studio', 'Apartment', 'House', 'Condo', 'Unique Stay']},
  {'Name': 'A', 'Value': ['Basic', 'Standard', 'Premium', 'Luxury', 'None']},
  {'Name': 'S',
   'Value': ['Peak', 'Off-Peak', 'Holiday', 'Weekend', 'Weekday']}],
 'Constraints': [{'Target': [{'Name': 'P',
     'Value': ['$0 - $50',
      '$51 - $100',
      '$101 - $200',
      '$201 - $500',
      '$501 and above']}],
   'Condition': [],
   'Probability': [0.1, 0.25, 0.35, 0.2, 0.1]},
  {'Target': [{'Name': 'G', 'Value': ['1', '2', '3-4', '5-6', '7+']}],
   'Condition': [],
   'Probability': [0.15, 0.4, 0.3, 0.1, 0.05]},
  {'Target': [{'Name': 'L',
     'Value': ['Beach',
      'City Center',
      'Suburban',
     

In [134]:
folder_path = './Dataset/bin1'
sub_folders = os.listdir(folder_path)
sub_folders
folds = []
for sub_folder in sub_folders:
    if not sub_folder.startswith('.'):
        folds.append(folder_path + '/' + sub_folder)
folds

['./Dataset/bin1/Rochester',
 './Dataset/bin1/Newark',
 './Dataset/bin1/BrowardCounty',
 './Dataset/bin1/SanDiego',
 './Dataset/bin1/Cambridge',
 './Dataset/bin1/SantaClaraCounty',
 './Dataset/bin1/Dallas',
 './Dataset/bin1/LosAngeles',
 './Dataset/bin1/Salem',
 './Dataset/bin1/Denver',
 './Dataset/bin1/FortWorth']

In [178]:
import concurrent.futures

def run_evaluation(fold):
    test = evaluation(repeat=2, verbose=False)
    test.run_all(fold, run_mrf=True)
    return test


# Use ThreadPoolExecutor for multithreading
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Map the run_evaluation function to each fold
    results = list(executor.map(run_evaluation, left))

# Now 'results' contains the results of all the evaluations


 16%|█▌        | 1594/10000 [00:07<00:54, 153.15it/s]

ERROR: Value not found 3+ Bedrooms
ERROR: Value not found Suburban
ERROR: Invalid input
ERROR: Value not found 3+ Bedrooms
ERROR: Invalid input
ERROR: invalid constraint type 3 {'Target': [{'Name': 'Bedrooms', 'Value': ['3+ Bedrooms']}], 'Condition': [{'Name': 'LocationProximity', 'Value': ['Suburban']}], 'Probability': [0.5]}
ERROR: Value not found Rural
ERROR: Invalid input
ERROR: Value not found 3+ Bedrooms
ERROR: Invalid input
ERROR: invalid constraint type 3 {'Target': [{'Name': 'Bedrooms', 'Value': ['3+ Bedrooms']}], 'Condition': [{'Name': 'LocationProximity', 'Value': ['Rural']}], 'Probability': [0.7]}
ERROR: Value not found Peak Season
ERROR: Invalid input
ERROR: Variable not found OccupancyRate
ERROR: Value not found Off-Peak Season
ERROR: Invalid input
ERROR: Variable not found OccupancyRate
ERROR: Value not found 2+ Bedrooms
ERROR: Value not found 2+ Bedrooms
ERROR: Variable not found GuestRating
ERROR: Variable not found GuestRating
ERROR: Duplicate constraints with  0
ERRO

 23%|██▎       | 2303/10000 [00:11<00:43, 178.63it/s]

 23%|██▎       | 2322/10000 [00:11<00:53, 143.96it/s]

 25%|██▌       | 2503/10000 [00:12<01:13, 102.26it/s]

 26%|██▌       | 2606/10000 [00:13<01:06, 111.07it/s]

 26%|██▋       | 2629/10000 [00:14<01:14, 98.99it/s] 

 26%|██▋       | 2640/10000 [00:14<01:57, 62.84it/s]


 26%|██▋       | 2648/10000 [00:14<02:00, 60.89it/s]


 27%|██▋       | 2657/10000 [00:14<01:51, 65.58it/s]


 27%|██▋       | 2665/10000 [00:14<01:49, 66.88it/s]


 27%|██▋       | 2673/10000 [00:14<01:46, 68.87it/s]

 27%|██▋       | 2683/10000 [00:15<01:37, 74.86it/s]



 27%|██▋       | 2691/10000 [00:15<01:37, 75.28it/s]


 27%|██▋       | 2702/10000 [00:15<01:27, 83.61it/s]


 27%|██▋       | 2713/10000 [00:15<01:22, 88.60it/s]


 27%|██▋       | 2724/10000 [00:15<01:17, 94.02it/s]

 27%|██▋       | 2734/10000 [00:15<01:17, 93.85it/s]


 27%|██▋       | 2744/10000 [00:15<01:23, 87.25it/s]


 28%|██▊       | 2753/10000 [00:15<01:27, 82.83it/s]


 28%|██▊   

ERROR: Duplicate constraints with  0
ERROR: Duplicate constraints with  1
ERROR: Duplicate constraints with  2
ERROR: Duplicate constraints with  3
ERROR: Duplicate constraints with  4





































 59%|█████▊    | 5858/10000 [02:24<06:16, 11.01it/s]












































 59%|█████▊    | 5860/10000 [02:24<05:57, 11.58it/s]





















































 59%|█████▊    | 5862/10000 [02:24<05:26, 12.66it/s]












































 59%|█████▊    | 5864/10000 [02:24<05:02, 13.68it/s]



































 59%|█████▊    | 5866/10000 [02:24<04:39, 14.77it/s]












































 59%|█████▊    | 5868/10000 [02:24<04:25, 15.57it/s]












































 59%|█████▊    | 5871/10000 [02:24<03:44, 18.42it/s]











































 59%|█████▊    | 5874/10000 [02:25<03:34, 19.26it/s]























































 59%|█████▉    | 5877/10000 [02:25<03:23, 20.30it/s]







































 59%|█████▉    | 5880/10000 [02:25<03:05, 22.21it/s]


































